P1 discontinuous SWIPDG, stationary linear elliptic ESV2007 problem
==================================

This example is about approximating the solution $u$ of the elliptic problem

$$\begin{align}
  -\nabla\cdot( \kappa \nabla u ) &= f   &&\text{in } \Omega\\
                                u &= g_D &&\text{on }\partial\Omega
\end{align}$$

with datafunction as defined in `dune/gdt/test/linearelliptic/problems/ESV2007.hh` (see below) using a piecewise linear block SWIPDG scheme, as in `dune/gdt/test/linearelliptic/discretizers/block-ipdg.hh`.

In [ ]:
from dune.xt.common import init_logger, init_mpi
init_mpi()

$$\begin{align}
  \Omega &= [-1, 1]^2\\
  \Gamma_D &= \partial\Omega\\
  \Gamma_N &= \emptyset
\end{align}$$

In [ ]:
print('initializing grid and problem ...')

from dune.xt.grid import (
    make_cube_dd_subdomains_grid__2d_simplex_aluconform as make_grid,
    make_boundary_info_on_dd_subdomain_layer as make_subdomain_boundary_info,
    make_boundary_info_on_dd_subdomain_boundary_layer as make_boundary_info
)

# The estimator notebook relies on this grid cfg!
inner_boundary_id=18446744073709551573
grid = make_grid(lower_left=[-1, -1], upper_right=[1, 1], num_elements=[4, 4], num_refinements=2,
                 num_partitions=[2, 2], num_oversampling_layers=2, inner_boundary_segment_index=inner_boundary_id)
grid.visualize('../block_swipdg_estimates_grid', with_coupling=True)

all_dirichlet_boundary_info = make_boundary_info(grid, {'type': 'xt.grid.boundaryinfo.alldirichlet'})
all_neumann_boundary_info = make_subdomain_boundary_info(grid, {'type': 'xt.grid.boundaryinfo.allneumann'})

$$\begin{align}\lambda(x) &:= 1\\
\kappa(x) &:= [1, 0; 0, 1] \in \mathbb{R}^{2\times 2}\\
f(x) &:= \tfrac{1}{2} \pi^2 \cos(\tfrac{1}{2} \pi x_0) \cos(\tfrac{1}{2} \pi x_1)\\
g_D(x) &:= 0\end{align}$$

Note that the `grid` is only provided to select the correct _type_ of function; these functions do not rely on the actual grid object.

In [ ]:
from dune.xt.functions import (
    make_checkerboard_function_1x1,
    make_constant_function_1x1,
    make_constant_function_2x2,
    make_expression_function_1x1
)

# The estimator notebook relies on this problem cfg!
lambda_ = make_constant_function_1x1(grid, 1.0, name='lambda')
kappa = make_constant_function_2x2(grid, [[1., 0.], [0., 1.]], name='kappa')
f = make_expression_function_1x1(grid, 'x', '0.5*pi*pi*cos(0.5*pi*x[0])*cos(0.5*pi*x[1])', order=2, name='f')

In [ ]:
print('preparing sparsity patterns ...')

from dune.xt.la import (
    IstlRowMajorSparseMatrixDouble as Matrix,
    IstlDenseVectorDouble as Vector
)
from dune.gdt import make_block_dg_dd_subdomain_part_to_1x1_fem_p1_space as make_block_space

block_space = make_block_space(grid)

local_patterns = [block_space.local_space(ii).compute_pattern('face_and_volume')
                  for ii in range(block_space.num_blocks)]
coupling_patterns_in_in = {}
coupling_patterns_out_out = {}
coupling_patterns_in_out = {}
coupling_patterns_out_in = {}
for ii in range(grid.num_subdomains):
    for jj in grid.neighboring_subdomains(ii):
        if ii < jj: # Assemble primally (visit each coupling only once).
            coupling_patterns_in_in[(ii, jj)] = block_space.local_space(ii).compute_pattern('volume')
            coupling_patterns_out_out[(ii, jj)] = block_space.local_space(jj).compute_pattern('volume')
            coupling_patterns_in_out[(ii, jj)] = block_space.compute_coupling_pattern(ii, jj, 'face')
            coupling_patterns_out_in[(ii, jj)] = block_space.compute_coupling_pattern(jj, ii, 'face')
boundary_patterns = {}
for ii in grid.boundary_subdomains():
    boundary_patterns[ii] = block_space.compute_boundary_pattern(ii, 'volume')

local_matrices = [None]*grid.num_subdomains
local_vectors = [None]*grid.num_subdomains
boundary_matrices = {}
coupling_matrices_in_in = {}
coupling_matrices_out_out = {}
coupling_matrices_in_out = {}
coupling_matrices_out_in = {}
for ii in range(grid.num_subdomains):
    local_matrices[ii] = Matrix(block_space.local_space(ii).size(),
                                block_space.local_space(ii).size(),
                                local_patterns[ii])
    local_vectors[ii] = Vector(block_space.local_space(ii).size())
    if ii in grid.boundary_subdomains():
        boundary_matrices[ii] = Matrix(block_space.local_space(ii).size(),
                                       block_space.local_space(ii).size(),
                                       boundary_patterns[ii])
    for jj in grid.neighboring_subdomains(ii):
        if ii < jj: # Assemble primally (visit each coupling only once).
            coupling_matrices_in_in[(ii, jj)] = Matrix(block_space.local_space(ii).size(),
                                              block_space.local_space(ii).size(),
                                              coupling_patterns_in_in[(ii, jj)])
            coupling_matrices_out_out[(ii, jj)] = Matrix(block_space.local_space(jj).size(),
                                                block_space.local_space(jj).size(),
                                                coupling_patterns_out_out[(ii, jj)])
            coupling_matrices_in_out[(ii, jj)] = Matrix(block_space.local_space(ii).size(),
                                               block_space.local_space(jj).size(),
                                               coupling_patterns_in_out[(ii, jj)])
            coupling_matrices_out_in[(ii, jj)] = Matrix(block_space.local_space(jj).size(),
                                               block_space.local_space(ii).size(),
                                               coupling_patterns_out_in[(ii, jj)])

In [ ]:
print('assembling locally ...')

from dune.xt.grid import (
    make_apply_on_dirichlet_intersections_dd_subdomain_boundary_part as make_apply_on_dirichlet_intersections
)

from dune.gdt import (
    make_elliptic_swipdg_affine_factor_matrix_operator as make_elliptic_swipdg_matrix_operator,
    make_l2_volume_vector_functional,
    make_local_elliptic_swipdg_affine_factor_inner_integral_operator_1x1_p1_dg_fem_space_dd_subdomain_coupling_intersection as make_local_elliptic_swipdg_coupling_operator,
    make_local_elliptic_swipdg_affine_factor_boundary_integral_operator_1x1_p1_dg_fem_space_dd_subdomain_coupling_intersection as make_local_elliptic_swipdg_boundary_operator,
    make_system_assembler
)

def assemble_local_contributions(subdomain):
    ipdg_operator = make_elliptic_swipdg_matrix_operator(lambda_, kappa, all_neumann_boundary_info,
                                                         local_matrices[subdomain],
                                                         block_space.local_space(subdomain))
    l2_functional = make_l2_volume_vector_functional(f, local_vectors[subdomain], block_space.local_space(subdomain))
    local_assembler = make_system_assembler(block_space.local_space(subdomain))
    local_assembler.append(ipdg_operator)
    local_assembler.append(l2_functional)
    local_assembler.assemble()    

for ii in range(grid.num_subdomains):
    assemble_local_contributions(ii)

local_ipdg_coupling_operator = make_local_elliptic_swipdg_coupling_operator(lambda_, kappa)

def assemble_coupling_contributions(subdomain, neighboring_subdomain):
    coupling_assembler = block_space.coupling_assembler(subdomain, neighboring_subdomain)
    coupling_assembler.append(local_ipdg_coupling_operator,
                              coupling_matrices_in_in[(subdomain, neighboring_subdomain)],
                              coupling_matrices_out_out[(subdomain, neighboring_subdomain)],
                              coupling_matrices_in_out[(subdomain, neighboring_subdomain)],
                              coupling_matrices_out_in[(subdomain, neighboring_subdomain)])
    coupling_assembler.assemble()
    
for ii in range(grid.num_subdomains):
    for jj in grid.neighboring_subdomains(ii):
        if ii < jj: # Assemble primally (visit each coupling only once).
            assemble_coupling_contributions(ii, jj)

local_ipdg_boundary_operator = make_local_elliptic_swipdg_boundary_operator(lambda_, kappa)
apply_on_dirichlet_intersections = make_apply_on_dirichlet_intersections(all_dirichlet_boundary_info)
            
def assemble_boundary_contributions(subdomain):
    boundary_assembler = block_space.boundary_assembler(subdomain)
    boundary_assembler.append(local_ipdg_boundary_operator,
                              boundary_matrices[subdomain],
                              apply_on_dirichlet_intersections)
    boundary_assembler.assemble()
    
for ii in grid.boundary_subdomains():
    assemble_boundary_contributions(ii)

In [ ]:
print('assembling globally ...')

from dune.xt.la import SparsityPatternDefault

global_pattern = SparsityPatternDefault(block_space.mapper.size)
for ii in range(grid.num_subdomains):
    block_space.mapper.copy_local_to_global(local_patterns[ii], ii, global_pattern)
    if ii in grid.boundary_subdomains():
        block_space.mapper.copy_local_to_global(boundary_patterns[ii], ii, global_pattern)
    for jj in grid.neighboring_subdomains(ii):
        if ii < jj: # Assemble primally (visit each coupling only once).
            block_space.mapper.copy_local_to_global(coupling_patterns_in_in[(ii, jj)], ii, ii, global_pattern)
            block_space.mapper.copy_local_to_global(coupling_patterns_out_out[(ii, jj)], jj, jj, global_pattern)
            block_space.mapper.copy_local_to_global(coupling_patterns_in_out[(ii, jj)], ii, jj, global_pattern)
            block_space.mapper.copy_local_to_global(coupling_patterns_out_in[(ii, jj)], jj, ii, global_pattern)
        
system_matrix = Matrix(block_space.mapper.size, block_space.mapper.size, global_pattern)
rhs_vector = Vector(block_space.mapper.size, 0.)
for ii in range(grid.num_subdomains):
    block_space.mapper.copy_local_to_global(local_matrices[ii], local_patterns[ii], ii, system_matrix)
    block_space.mapper.copy_local_to_global(local_vectors[ii], ii, rhs_vector)
    if ii in grid.boundary_subdomains():
        block_space.mapper.copy_local_to_global(boundary_matrices[ii], boundary_patterns[ii], ii, ii, system_matrix)
    for jj in grid.neighboring_subdomains(ii):
        if ii < jj: # Assemble primally (visit each coupling only once).
            block_space.mapper.copy_local_to_global(coupling_matrices_in_in[(ii, jj)],
                                                    coupling_patterns_in_in[(ii, jj)],
                                                    ii, ii, system_matrix)
            block_space.mapper.copy_local_to_global(coupling_matrices_out_out[(ii, jj)],
                                                    coupling_patterns_out_out[(ii, jj)],
                                                    jj, jj, system_matrix)
            block_space.mapper.copy_local_to_global(coupling_matrices_in_out[(ii, jj)],
                                                    coupling_patterns_in_out[(ii, jj)],
                                                    ii, jj, system_matrix)
            block_space.mapper.copy_local_to_global(coupling_matrices_out_in[(ii, jj)],
                                                    coupling_patterns_out_in[(ii, jj)],
                                                    jj, ii, system_matrix)

In [ ]:
print('solving ...')

from dune.xt.la import make_solver
from dune.gdt import make_discrete_function

u_h_vector = Vector(block_space.mapper.size, 0.0)
make_solver(system_matrix).apply(rhs_vector, u_h_vector)
u_h = make_discrete_function(block_space, u_h_vector, 'u_h')
u_h.visualize('../block_swipdg_esv2007_solution')